In [4]:
import torch
from transformers import AutoTokenizer

In [6]:
train_path = 'data/20_000_examples_train.pt'
eval_path = 'data/20_000_examples_eval.pt'
tokenizer = AutoTokenizer.from_pretrained('AI-Sweden-Models/gpt-sw3-6.7b')

train_data = torch.load(train_path)
eval_data = torch.load(eval_path)



39.708333333333336
